In [1]:
import pandas as pd

In [2]:
Dataset=pd.read_csv('CKD.csv')

In [3]:
Dataset.head()

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,2.0,76.459948,c,3,0,normal,abnormal,notpresent,notpresent,148.112676,...,38.868902,8408.191126,4.705597,no,no,no,yes,yes,no,yes
1,3.0,76.459948,c,2,0,normal,normal,notpresent,notpresent,148.112676,...,34.000000,12300.000000,4.705597,no,no,no,yes,poor,no,yes
2,4.0,76.459948,a,1,0,normal,normal,notpresent,notpresent,99.000000,...,34.000000,8408.191126,4.705597,no,no,no,yes,poor,no,yes
3,5.0,76.459948,d,1,0,normal,normal,notpresent,notpresent,148.112676,...,38.868902,8408.191126,4.705597,no,no,no,yes,poor,yes,yes
4,5.0,50.000000,c,0,0,normal,normal,notpresent,notpresent,148.112676,...,36.000000,12400.000000,4.705597,no,no,no,yes,poor,no,yes


In [4]:
Dataset.columns

Index(['age', 'bp', 'sg', 'al', 'su', 'rbc', 'pc', 'pcc', 'ba', 'bgr', 'bu',
       'sc', 'sod', 'pot', 'hrmo', 'pcv', 'wc', 'rc', 'htn', 'dm', 'cad',
       'appet', 'pe', 'ane', 'classification'],
      dtype='object')

In [5]:
Dataset=pd.get_dummies(Dataset,dtype=int,drop_first=True)

In [6]:
Dataset.columns

Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [7]:
Independent=Dataset[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes',]]
Dependent=Dataset[['classification_yes']]

In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(Independent,Dependent,test_size=0.30, random_state=0)

In [14]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

param_grid = {
    'solver': ['lbfgs', 'liblinear', 'newton-cg', 'sag', 'saga'],  
    'penalty': ['l1', 'l2', 'elasticnet', 'none'],  
    'multi_class': ['auto', 'ovr', 'multinomial']
}

# Initialize GridSearchCV with LogisticRegression
grid = GridSearchCV(LogisticRegression(max_iter=1000), param_grid, refit=True, verbose=3, n_jobs=-1, scoring='f1_weighted')

# Fit the grid search
grid.fit(X_train, Y_train)

Fitting 5 folds for each of 60 candidates, totalling 300 fits


c:\Users\Gajalakshmi G\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
205 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Gajalakshmi G\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Gajalakshmi G\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  F

GridSearchCV(estimator=LogisticRegression(max_iter=1000), n_jobs=-1,
             param_grid={'multi_class': ['auto', 'ovr', 'multinomial'],
                         'penalty': ['l1', 'l2', 'elasticnet', 'none'],
                         'solver': ['lbfgs', 'liblinear', 'newton-cg', 'sag',
                                    'saga']},
             scoring='f1_weighted', verbose=3)

In [16]:
re=grid.cv_results_


In [17]:
grid_Pred=grid.predict(X_test)

In [18]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(grid_Pred,Y_test)
print(cm)

[[43  0]
 [ 2 75]]


In [19]:
from sklearn.metrics import classification_report
clf_report=classification_report(grid_Pred,Y_test)
print(clf_report)

              precision    recall  f1-score   support

           0       0.96      1.00      0.98        43
           1       1.00      0.97      0.99        77

    accuracy                           0.98       120
   macro avg       0.98      0.99      0.98       120
weighted avg       0.98      0.98      0.98       120



In [20]:
from sklearn.metrics import f1_score
f1_macro=f1_score(grid_Pred,Y_test)
print(f1_macro)

0.9868421052631579


In [21]:
from sklearn.metrics import roc_auc_score
roc_auc_score(Y_test,grid.predict_proba(X_test)[:,1])

0.9988148148148148

In [22]:
table=pd.DataFrame.from_dict(re)
print(table)

    mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0        0.002958  6.944852e-04         0.000000        0.000000   
1        0.028874  8.532455e-03         0.016666        0.004334   
2        0.003440  2.039651e-03         0.000000        0.000000   
3        0.002392  4.883444e-04         0.000000        0.000000   
4        0.303196  4.236820e-02         0.026433        0.021056   
5        0.264000  2.231947e-02         0.013204        0.002745   
6        0.016616  5.026946e-03         0.015865        0.006220   
7        0.155820  1.894079e-02         0.011192        0.002368   
8        0.171407  1.315318e-02         0.011460        0.000543   
9        0.192866  1.886285e-02         0.012844        0.001242   
10       0.002217  3.892086e-04         0.000000        0.000000   
11       0.003308  3.429544e-03         0.000000        0.000000   
12       0.002168  6.811439e-04         0.000000        0.000000   
13       0.002343  4.349354e-04         0.000000